# SW중심대학 공동 AI 경진대회 <본선>

1. 평가 산식

> - Accuracy (정확도)

2. 본선 평가

![](https://dacon.s3.ap-northeast-2.amazonaws.com/competition/235970/editor-image/166138683752218.jpeg)

3. 개인 또는 팀 참여 규칙

> - 본선 진출자는 예선과 동일한 팀원 및 팀명으로 참가해야 함
> - 본선 진출자 전원은 9/16(금)까지 재학증명서 또는 휴학증명서를 dacon@dacon.io 메일로 제출하야 함 [링크]
> ※ 허위 참가사실이 확인되거나 혹은 기간내 서류 제출을 하지 않을 시, 소속 팀 전체가 대회 탈락 처리됨을 유의 부탁드립니다.
> ※ 또한, 대회 기간 도중 팀원 중 졸업생 신분의 참가자가 포함되어있는 것이 확인될 경우에도 해당 팀 전체가 대회 탈락 처리됩니다.

 

4. 외부 데이터 및 사전 학습 모델 사용

> - 비상업적 용도로 사용할 수 있는 외부 데이터 사용 가능
> - 사용에 문제가 없는 사전 학습 모델(Pre-trained Model) 사용 가능


5. 유의 사항

> - 1일 최대 제출 횟수: 3회
> - 사용 가능 언어: Python, R
> - 모델 학습에서 평가 데이터셋(Test Dataset) 활용(Data Leakage) 시 실격 처리 됨 (참조 : 링크)
> - 답안을 수기로 작성하는 경우 실격 처리 됨
> - 다른 팀과의 아이디어 또는 코드 쉐어링이 의심되는 경우 데이콘에서 코드를 요청할 수 있으며, 기간 내 코드를 제출하지 않거나 치팅이 확인되는 경우 실격 처리 됨
> - 재학/휴학 증명서, 발표 자료 및 코드를 요청한 일자에 제출하지 않은 경우 실격 처리 됨
> - 최종 순위는 선택된 파일 중에서 채점되므로 참가자는 제출 창에서 자신이 최종적으로 채점을 받고 싶은 파일을 선택해야 함
> - 소프트웨어중심대학 공동 AI 경진대회 추진위원회는 대회 참가팀의 부정 행위를 금지하고 있으며, 이와 관련하여 본 경기를 위탁 운영하는 데이콘의 다른 경진대회에서 부정 제출 이력이 있는 경우에는 평가가 제한됩니다.
> - 자세한 사항은 아래의 링크를 참고해 주시기 바랍니다. https://dacon.io/notice/notice/13

6. 토론(질문)

> - 대회 운영 및 데이터 이상에 관련된 질문 외에는 답변을 드리지 않습니다.
> - 기타 질문은 토론 페이지를 통해 자유롭게 토론해주시기 바랍니다.
> - 소프트웨어중심대학 공동 AI 경진대회 추진위원회의 답변을 희망하시는 경우에는 토크 게시판의 대회 문의 게시글에 댓글을 달아주시면 검토하여 답변을 드리겠습니다.


## 모듈 및 설정

In [1]:
import os
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from jamo import h2j, j2hcj
from join_jamos import join_jamos
from trdg.generators import GeneratorFromStrings
from sklearn.model_selection import train_test_split

Missing modules for handwritten text generation.


In [200]:
def save_gt(filename, img_path_list, text_list):
    with open(filename, 'w', encoding='utf-8') as f:
        for img_path, text in zip(img_path_list, text_list):
            f.write(f'{img_path}\t{j2hcj(h2j(text))}\n')

In [222]:
def data_generate(char, fonts_list, count=1, orientation=0):
    lengths = np.random.randint(1, 10, count)
    strings = [''.join(list(np.random.choice(char, length))) for length in lengths]
    fonts = np.random.choice(fonts_list, count)
    size = int(np.random.normal(300, 50))
    blur = np.random.choice([True, False])
    background_type = np.random.choice([0, 1, 2])
    distorsion_type = np.random.choice([0, 1, 2, 3])
    distorsion_orientation = np.random.choice([0, 1, 2])
    text_color = f'#{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}'
    character_spacing = np.random.randint(0, 20) * np.random.randint(0, 20)
    skewing_angle = int((400 - character_spacing) * 45 / 400)
    margins = (
        size * np.random.randint(-5, 15) // 100,
        size * np.random.randint(-5, 15) // 100,
        size * np.random.randint(-5, 15) // 100,
        size * np.random.randint(-5, 15) // 100
    )
    stroke_width = np.random.choice([0, 0, 0, 3, 6])
    stroke_fill = f'#{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}'

    generator = GeneratorFromStrings(
        strings,
        fonts=fonts,
        count=count,
        language='ko',
        size=size,
        skewing_angle=skewing_angle,
        random_skew=True,
        blur=blur,
        background_type=background_type,
        distorsion_type=distorsion_type,
        distorsion_orientation=distorsion_orientation,
        text_color=text_color,
        orientation=orientation,
        character_spacing=character_spacing,
        margins=margins,
        stroke_width=stroke_width,
        stroke_fill=stroke_fill,
    )

    img_list = []
    lbl_list = []

    for img, lbl in generator:
        img_list.append(np.array(img))
        lbl_list.append(lbl)

    return img_list, lbl_list

## 데이터

### 폴더 및 파일명

In [192]:
inputs = 'inputs'
outputs = 'outputs'
processing = 'processing'

train_csv = 'train.csv'
test_csv = 'test.csv'
submission_csv = 'sample_submission.csv'

separation = 'separation'
lmdb = 'lmdb'
generate = 'generate'

horizontal = 'horizontal'
vertical = 'vertical'

train_gt = 'train_gt.txt'
test_gt = 'test_gt.txt'
training_gt = 'training_gt.txt'
validation_gt = 'validation_gt.txt'
generate_gt = 'generate_gt.txt'

### CSV 파일 가져오기

In [13]:
train = pd.read_csv(os.path.join(inputs, train_csv))
test = pd.read_csv(os.path.join(inputs, test_csv))
submission = pd.read_csv(os.path.join(outputs, submission_csv))

### 새로운 텍스트 생성

In [201]:
first = [*'ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ']
middle = [*'ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ']
last = [''] + [*'ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ']

ko = [join_jamos(f'{f}{m}{l}') for f in first for m in middle for l in last]
fonts_list = list(map(lambda x : os.path.join('trdg/fonts/ko/', x), os.listdir('trdg/fonts/ko/')))

In [224]:
img_list, lbl_list = [], []
img_path_list, text_list = [], []
idx = 0

for _ in tqdm(range(10000)):
    img_tmp, lbl_tmp = data_generate(ko, fonts_list, orientation=1)

    for img, lbl in zip(img_tmp, lbl_tmp):
        img_path = f'./{generate}/{idx:06d}.png'
        if img.ndim != 0:
            os.makedirs(os.path.dirname(os.path.join(processing, generate, horizontal, img_path)), exist_ok = True)
            cv2.imwrite(os.path.join(processing, generate, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
            img_path_list.append(img_path)
            text_list.append(lbl)
            idx = idx + 1
    
save_gt(os.path.join(processing, generate, horizontal, generate_gt), img_path_list, text_list)

100%|██████████| 10000/10000 [2:53:11<00:00,  1.04s/it] 


In [223]:
img_list, lbl_list = [], []
img_path_list, text_list = [], []
idx = 0

for _ in tqdm(range(10000)):
    img_tmp, lbl_tmp = data_generate(ko, fonts_list, orientation=0)

    for img, lbl in zip(img_tmp, lbl_tmp):
        img_path = f'./{generate}/{idx:06d}.png'
        if img.ndim != 0:
            os.makedirs(os.path.dirname(os.path.join(processing, generate, vertical, img_path)), exist_ok = True)
            cv2.imwrite(os.path.join(processing, generate, vertical, img_path), img)
            img_path_list.append(img_path)
            text_list.append(lbl)
            idx = idx + 1
    
save_gt(os.path.join(processing, generate, vertical, generate_gt), img_path_list, text_list)

100%|██████████| 10000/10000 [2:25:30<00:00,  1.15it/s] 


### 모델 튜닝 및 최종 학습 데이터셋 생성

In [38]:
horizontal_img_path = []
horizontal_text = []
vertical_img_path = []
vertical_text = []

for img_path, text in zip(train['img_path'], train['text']):
    img = cv2.imread(os.path.join(inputs, img_path))
    height, weight, color = img.shape
    if height > weight:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, horizontal, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        horizontal_img_path.append(img_path)
        horizontal_text.append(text)
    else: 
        os.makedirs(os.path.dirname(os.path.join(processing, separation, vertical, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, vertical, img_path), img)
        vertical_img_path.append(img_path)
        vertical_text.append(text)

# Horizontal
save_gt(os.path.join(processing, separation, horizontal, train_gt), horizontal_img_path, horizontal_text)

X_train, X_valid, y_train, y_valid = train_test_split(horizontal_img_path, horizontal_text, test_size=0.2)

save_gt(os.path.join(processing, separation, horizontal, training_gt), X_train, y_train)
save_gt(os.path.join(processing, separation, horizontal, validation_gt), X_valid, y_valid)

# Vertical
save_gt(os.path.join(processing, separation, vertical, train_gt), vertical_img_path, vertical_text)

X_train, X_valid, y_train, y_valid = train_test_split(vertical_img_path, vertical_text, test_size=0.2)

save_gt(os.path.join(processing, separation, vertical, training_gt), X_train, y_train)
save_gt(os.path.join(processing, separation, vertical, validation_gt), X_valid, y_valid)


### 최종 예측 데이터셋 생성

In [15]:
horizontal_img_path = []
vertical_img_path = []

for img_path in test['img_path']:
    img = cv2.imread(os.path.join(inputs, img_path))
    height, weight, color = img.shape
    if height > weight:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, horizontal, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        horizontal_img_path.append(img_path)
    else:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, vertical, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, vertical, img_path), img)
        vertical_img_path.append(img_path)

horizontal_text = ['예측' for _ in range(len(horizontal_img_path))]
vertical_text = ['예측' for _ in range(len(vertical_img_path))]

# Horizontal
save_gt(os.path.join(processing, separation, horizontal, test_gt), horizontal_img_path, horizontal_text)

# Vertical
save_gt(os.path.join(processing, separation, vertical, test_gt), vertical_img_path, vertical_text)

### LMDB 데이터셋 생성

In [1]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/horizontal --gtFile processing/separation/horizontal/train_gt.txt --outputPath processing/lmdb/horizontal/train

Written 1000 / 1985
Created dataset with 1985 samples


In [2]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/vertical --gtFile processing/separation/vertical/train_gt.txt --outputPath processing/lmdb/vertical/train

Written 1000 / 10174
Written 2000 / 10174
Written 3000 / 10174
Written 4000 / 10174
Written 5000 / 10174
Written 6000 / 10174
Written 7000 / 10174
Written 8000 / 10174
Written 9000 / 10174
Written 10000 / 10174
Created dataset with 10174 samples


In [16]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/horizontal --gtFile processing/separation/horizontal/test_gt.txt --outputPath processing/lmdb/horizontal/test

Created dataset with 445 samples


In [17]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/vertical --gtFile processing/separation/vertical/test_gt.txt --outputPath processing/lmdb/vertical/test

Written 1000 / 3224
Written 2000 / 3224
Written 3000 / 3224
Created dataset with 3224 samples


In [5]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/horizontal --gtFile processing/separation/horizontal/training_gt.txt --outputPath processing/lmdb/horizontal/training

Written 1000 / 1588
Created dataset with 1588 samples


In [6]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/vertical --gtFile processing/separation/vertical/training_gt.txt --outputPath processing/lmdb/vertical/training

Written 1000 / 8139
Written 2000 / 8139
Written 3000 / 8139
Written 4000 / 8139
Written 5000 / 8139
Written 6000 / 8139
Written 7000 / 8139
Written 8000 / 8139
Created dataset with 8139 samples


In [7]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/horizontal --gtFile processing/separation/horizontal/validation_gt.txt --outputPath processing/lmdb/horizontal/validation

Created dataset with 397 samples


In [8]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/separation/vertical --gtFile processing/separation/vertical/validation_gt.txt --outputPath processing/lmdb/vertical/validation

Written 1000 / 2035
Written 2000 / 2035
Created dataset with 2035 samples


In [226]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/generate/horizontal --gtFile processing/generate/horizontal/generate_gt.txt --outputPath processing/lmdb/horizontal/generate

Written 1000 / 9403
Written 2000 / 9403
Written 3000 / 9403
Written 4000 / 9403
Written 5000 / 9403
Written 6000 / 9403
Written 7000 / 9403
Written 8000 / 9403
Written 9000 / 9403
Created dataset with 9403 samples


In [227]:
!python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath processing/generate/vertical --gtFile processing/generate/vertical/generate_gt.txt --outputPath processing/lmdb/vertical/generate

Written 1000 / 9419
Written 2000 / 9419
Written 3000 / 9419
Written 4000 / 9419
Written 5000 / 9419
Written 6000 / 9419
Written 7000 / 9419
Written 8000 / 9419
Written 9000 / 9419
Created dataset with 9419 samples


## 모델

### deep_text_recognition_benchmark

아래의 명령어를 실행해 수평 / 수직 방향에 대한 간판이미지를 따로 학습   
(메모리 누수 문제로 인해서 Jupyter notebook 안에서 실행하지 않음)

python deep_text_recognition_benchmark/train.py --train_data processing/lmdb/horizontal/ --select_data training-generate --batch_ratio 0.5-0.5 --valid_data processing/lmdb/horizontal/validation --batch_size 20 --manualSeed 16 --imgH 64 --imgW 256 --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --num_fiducial 20 --output_channel 1024 --hidden_size 1024

python deep_text_recognition_benchmark/train.py --train_data processing/lmdb/horizontal/ --select_data training-generate --batch_ratio 0.5-0.5 --valid_data processing/lmdb/vertical/validation --batch_size 20 --manualSeed 15 --imgH 64 --imgW 256 --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --num_fiducial 20 --output_channel 1024 --hidden_size 1024